# 2025 Kaggle Competition of AI Applied to Medicine at UC3M





Welcome to the **2025 Kaggle competition of AI applied to medicine at UC3M**. This project is set up as an **internal Kaggle competition** in which all students will participate. Our real-world challenge for this course will revolve around the **ISIC 2024** dataset, a large collection of skin images used for research in dermatology.


Welcome to our simple **ResNet50-based** starter notebook. Below we:
1. **Define** a function to load images from HDF5 files.
2. **Load** and display our training metadata (no preprocessing).
3. **Load** a pretrained **ResNet50** model (we won't fine-tune it).
4. **Evaluate** test samples (in a trivial way for demonstration).
5. **Generate** a simple `submission.csv` file with the required format.

> **Note**: This is a minimal example to help you set up your environment. It doesn’t include any real training or meaningful model inference. Feel free to modify it to perform actual classification (e.g., add custom layers, train on your dataset, etc.).

## ISIC 2024 Competition Overview

The **International Skin Imaging Collaboration (ISIC)** has launched this competition to advance automated skin cancer detection by:

- **Improving accuracy** in distinguishing malignant from benign lesions
- **Enhancing efficiency** in clinical workflows
- **Developing algorithms** that prioritize high-risk lesions
- **Reducing mortality rates** by enabling earlier detection

### Primary Task

You need to **classify skin lesions** as **benign** or **malignant**. For each lesion image (identified by `isic_id`), predict a **probability** in the range [0, 1] indicating the chance that the lesion is malignant.

### High-Level Data Summary

- The dataset is called **SLICE-3D**, containing **skin lesion images** (JPEG files) cropped from 3D Total Body Photography (TBP).
- Each image has metadata in a corresponding `.csv` file, including:
  - **Binary diagnostic label** (`target` = 0 or 1)
  - **Patient data** (e.g., `age_approx`, `sex`, `anatom_site_general`)
  - **Additional attributes** (image source, diagnosis type)

![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F4972760%2F349a3ae1149d15dc5642063a2d742c88%2Fimage%20type_noexif_240425.jpg?generation=1714060307710359&alt=media)

This challenge dataset mimics **non-dermoscopic images** using standardized 15x15 mm “tiles” of lesions from a 3D TBP system. Thousands of patients from multiple continents are represented, creating a broad, diverse dataset.

### Task Description & Clinical Context

- **Why it matters**: Skin cancer can be deadly if not detected early. Many people lack access to dermatologic care, so accurate AI systems for image-based triage can improve outcomes.
- **Key goal**: Develop a binary classifier that identifies malignant lesions from a set of smartphone-quality images.
- **Impact**: This technology could help prioritize suspicious lesions (top K) for clinical review, especially in low-resource settings, potentially **saving lives** through earlier detection.

### Importance of 3D TBP

The **3D Total Body Photography (TBP)** approach captures the entire skin surface in macro resolution. Each lesion on the patient’s body is automatically cropped as a 15x15 mm image tile. These images more closely resemble photos taken by a regular smartphone camera, as opposed to specialized dermoscopy devices.

![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F4972760%2F169b1f691322233e7b31aabaf6716ff3%2Fex-tiles.png?generation=1717700538524806&alt=media)

### Clinical Background

1. **Major skin cancer types**: Basal Cell Carcinoma (BCC), Squamous Cell Carcinoma (SCC), and Melanoma (most lethal).
2. **Early detection** is crucial: Minor surgery can cure many skin cancers if caught in time.
3. **Telemedicine implications**: With the rise in remote healthcare, patients often submit low-quality images captured at home. Robust AI models are needed to handle this variability.

### Summary

- You will build a model to **classify skin lesions** (benign vs. malignant) with probabilities.
- The dataset includes **every lesion** from thousands of patients, reflecting real-world diversity.
- **3D TBP** and the “ugly duckling sign” concept illustrate the importance of comparing each lesion against the patient’s total lesion landscape.
- **Your work** can help improve early detection, prioritizing high-risk cases for clinical evaluation and potentially saving lives.


In [ ]:
METADATA_COL2DESC = {
    "isic_id": "Unique identifier for each image case.",
    "target": "Binary class label (0 = benign, 1 = malignant).",
    "patient_id": "Unique identifier for each patient.",
    "age_approx": "Approximate age of the patient at time of imaging.",
    "sex": "Sex of the patient (male or female).",
    "anatom_site_general": "General location of the lesion on the patient's body.",
    "clin_size_long_diam_mm": "Maximum diameter of the lesion (mm).",
    "image_type": "Type of image captured, as defined in the ISIC Archive.",
    "tbp_tile_type": "Lighting modality of the 3D Total Body Photography (TBP) source image.",
    "tbp_lv_A": "Color channel A (green-red axis in LAB space) inside the lesion.",
    "tbp_lv_Aext": "Color channel A outside the lesion.",
    "tbp_lv_B": "Color channel B (blue-yellow axis in LAB space) inside the lesion.",
    "tbp_lv_Bext": "Color channel B outside the lesion.",
    "tbp_lv_C": "Chroma value inside the lesion.",
    "tbp_lv_Cext": "Chroma value outside the lesion.",
    "tbp_lv_H": "Hue value inside the lesion (LAB color space).",
    "tbp_lv_Hext": "Hue value outside the lesion.",
    "tbp_lv_L": "Luminance inside the lesion (LAB color space).",
    "tbp_lv_Lext": "Luminance outside the lesion.",
    "tbp_lv_areaMM2": "Area of the lesion in mm².",
    "tbp_lv_area_perim_ratio": "Ratio of the lesion's perimeter to its area (border jaggedness).",
    "tbp_lv_color_std_mean": "Mean color irregularity within the lesion.",
    "tbp_lv_deltaA": "Average contrast in color channel A between inside and outside.",
    "tbp_lv_deltaB": "Average contrast in color channel B between inside and outside.",
    "tbp_lv_deltaL": "Average contrast in luminance between inside and outside.",
    "tbp_lv_deltaLB": "Combined contrast between the lesion and surrounding skin.",
    "tbp_lv_deltaLBnorm": "Normalized contrast (LAB color space).",
    "tbp_lv_eccentricity": "Eccentricity of the lesion (how elongated it is).",
    "tbp_lv_location": "Detailed anatomical location (e.g., Upper Arm).",
    "tbp_lv_location_simple": "Simplified anatomical location (e.g., Arm).",
    "tbp_lv_minorAxisMM": "Smallest diameter of the lesion in mm.",
    "tbp_lv_nevi_confidence": "Confidence score (0-100) for the lesion being a nevus.",
    "tbp_lv_norm_border": "Normalized border irregularity (0-10 scale).",
    "tbp_lv_norm_color": "Normalized color variation (0-10 scale).",
    "tbp_lv_perimeterMM": "Perimeter of the lesion in mm.",
    "tbp_lv_radial_color_std_max": "Color asymmetry within the lesion, measured radially.",
    "tbp_lv_stdL": "Std. deviation of luminance inside the lesion.",
    "tbp_lv_stdLExt": "Std. deviation of luminance outside the lesion.",
    "tbp_lv_symm_2axis": "Asymmetry about a second axis of symmetry.",
    "tbp_lv_symm_2axis_angle": "Angle of that second axis of symmetry.",
    "tbp_lv_x": "X-coordinate in the 3D TBP model.",
    "tbp_lv_y": "Y-coordinate in the 3D TBP model.",
    "tbp_lv_z": "Z-coordinate in the 3D TBP model.",
    "attribution": "Image source or institution.",
    "copyright_license": "License information.",
    "lesion_id": "Unique ID for lesions of interest.",
    "iddx_full": "Full diagnosis classification.",
    "iddx_1": "First-level (broad) diagnosis.",
    "iddx_2": "Second-level diagnosis.",
    "iddx_3": "Third-level diagnosis.",
    "iddx_4": "Fourth-level diagnosis.",
    "iddx_5": "Fifth-level diagnosis.",
    "mel_mitotic_index": "Mitotic index of invasive malignant melanomas.",
    "mel_thick_mm": "Thickness of melanoma invasion in mm.",
    "tbp_lv_dnn_lesion_confidence": "Lesion confidence score (0-100) from a DNN classifier.",
}

METADATA_COL2NAME = {
    "isic_id": "Unique Case Identifier",
    "target": "Binary Lesion Classification",
    "patient_id": "Unique Patient Identifier",
    "age_approx": "Approximate Age",
    "sex": "Sex",
    "anatom_site_general": "General Anatomical Location",
    "clin_size_long_diam_mm": "Clinical Size (Longest Diameter in mm)",
    "image_type": "Image Type",
    "tbp_tile_type": "TBP Tile Type",
    "tbp_lv_A": "Color Channel A (Inside)",
    "tbp_lv_Aext": "Color Channel A (Outside)",
    "tbp_lv_B": "Color Channel B (Inside)",
    "tbp_lv_Bext": "Color Channel B (Outside)",
    "tbp_lv_C": "Chroma (Inside)",
    "tbp_lv_Cext": "Chroma (Outside)",
    "tbp_lv_H": "Hue (Inside)",
    "tbp_lv_Hext": "Hue (Outside)",
    "tbp_lv_L": "Luminance (Inside)",
    "tbp_lv_Lext": "Luminance (Outside)",
    "tbp_lv_areaMM2": "Lesion Area (mm²)",
    "tbp_lv_area_perim_ratio": "Area-to-Perimeter Ratio",
    "tbp_lv_color_std_mean": "Mean Color Irregularity",
    "tbp_lv_deltaA": "Delta A",
    "tbp_lv_deltaB": "Delta B",
    "tbp_lv_deltaL": "Delta L",
    "tbp_lv_deltaLB": "Delta LB",
    "tbp_lv_deltaLBnorm": "Normalized Delta LB",
    "tbp_lv_eccentricity": "Eccentricity",
    "tbp_lv_location": "Detailed Location",
    "tbp_lv_location_simple": "Simplified Location",
    "tbp_lv_minorAxisMM": "Smallest Diameter (mm)",
    "tbp_lv_nevi_confidence": "Nevus Confidence Score",
    "tbp_lv_norm_border": "Normalized Border Irregularity",
    "tbp_lv_norm_color": "Normalized Color Variation",
    "tbp_lv_perimeterMM": "Lesion Perimeter (mm)",
    "tbp_lv_radial_color_std_max": "Radial Color Deviation",
    "tbp_lv_stdL": "Std. Dev. Luminance (Inside)",
    "tbp_lv_stdLExt": "Std. Dev. Luminance (Outside)",
    "tbp_lv_symm_2axis": "Symmetry (Second Axis)",
    "tbp_lv_symm_2axis_angle": "Symmetry Angle (Second Axis)",
    "tbp_lv_x": "X-Coordinate",
    "tbp_lv_y": "Y-Coordinate",
    "tbp_lv_z": "Z-Coordinate",
    "attribution": "Image Source",
    "copyright_license": "Copyright",
    "lesion_id": "Unique Lesion ID",
    "iddx_full": "Full Diagnosis",
    "iddx_1": "Diagnosis Level 1",
    "iddx_2": "Diagnosis Level 2",
    "iddx_3": "Diagnosis Level 3",
    "iddx_4": "Diagnosis Level 4",
    "iddx_5": "Diagnosis Level 5",
    "mel_mitotic_index": "Mitotic Index (Melanoma)",
    "mel_thick_mm": "Melanoma Thickness (mm)",
    "tbp_lv_dnn_lesion_confidence": "Lesion DNN Confidence",
}

In [ ]:
import os
import h5py
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import matplotlib.pyplot as plt

from torchvision import models
from tqdm.auto import tqdm

from imblearn.over_sampling import SMOTE
import albumentations
from sklearn.preprocessing import LabelEncoder

# If using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# ---------------------------
# 1. Custom Dataset for HDF5
# ---------------------------
class ISIC_HDF5_Dataset(Dataset):
    """
    A PyTorch Dataset that loads images from an HDF5 file given a DataFrame of IDs.
    Applies image transforms suitable for ResNet50.
    """

    def __init__(
        self,
        df: pd.DataFrame,
        hdf5_path: str,
        transform=None,
        is_labelled: bool = True,
        balanced=False,
    ):
        """
        Args:
            df (pd.DataFrame): DataFrame containing 'isic_id' and optionally 'target'.
            hdf5_path (str): Path to the HDF5 file containing images.
            transform (callable): Optional transforms to be applied on a sample.
            is_labelled (bool): Whether the dataset includes labels (for train/val).
        """
        self.df = df.reset_index(drop=True)
        self.hdf5_path = hdf5_path
        self.transform = transform
        self.is_labelled = is_labelled
        self.balanced = balanced

        if self.balanced and self.is_labelled:
            self._balance_dataset()

    def _balance_dataset(self):  ## Added this function
        """Function to balance the dataset using SMOTE also converting non-numerical columns to numerical using LabelEncoder to be able to do the balancing."""
        # Store the original DataFrame
        original_df = self.df.copy()
        original_isic = self.df["isic_id"].tolist()

        # Augment images to balance the dataset
        self._augment_images()

        # Obtain a list of all new isic_id (added)
        new_isic_ids = [
            row[1]["isic_id"]
            for row in self.df.iterrows()
            if "_aug_" in row[1]["isic_id"]
        ]
        numerical_isic = [int(item.split("_")[1]) for item in original_isic]

        # Restore the original DataFrame
        self.df = original_df

        # Convert isiic_id to numerical
        self.df.loc[:, "isic_id"] = numerical_isic
        self.df["isic_id"] = self.df["isic_id"].astype("int64")

        non_numerical_data = self.df.select_dtypes(include=["object"])

        label_encoders = {}
        for column in non_numerical_data.columns:
            le = LabelEncoder()
            self.df[column] = le.fit_transform(self.df[column].astype(str))
            label_encoders[column] = le

        # Fill NaN values in numerical columns with the mean
        for column in self.df.select_dtypes(include=["float64", "int64"]).columns:
            self.df[column].fillna(self.df[column].mean(), inplace=True)

        # Fill NaN values in categorical columns with the mode
        for column in self.df.select_dtypes(include=["object"]).columns:
            self.df[column].fillna(self.df[column].mode()[0], inplace=True)

        # Separate features and target

        X = self.df.drop(columns=["target"])  # Excluding target from dataset
        y = self.df["target"]

        # Apply SMOTE
        smote = SMOTE(random_state=42)
        X_res, y_res = smote.fit_resample(X, y)

        # Create a new DataFrame with balanced data
        balanced_df = pd.DataFrame(X_res, columns=X.columns)
        balanced_df["target"] = y_res

        # Update the DataFrame
        self.df = balanced_df.reset_index(drop=True)

        def format_isic_id(x):
            x_str = str(x)  # Convert to string
            if len(x_str) < 7:  # If length is less than 7, pad with leading zeros
                x_str = x_str.zfill(7)  # zfill pads with leading zeros
            return "ISIC_" + x_str

        self.df["isic_id"] = self.df["isic_id"].apply(
            format_isic_id
        )  # Convert back to ISIC format
        self.df["patient_id"] = self.df["patient_id"].apply(
            lambda x: "IP_" + str(x)
        )  # Convert back to patient ID format

        # Change the isic_id of the augmented images to the new_isic_id to match with the images
        for idx, row in self.df.iterrows():
            if row["isic_id"] not in original_isic and row["target"] == 1:
                new_isic_id = new_isic_ids.pop(0)
                self.df.at[idx, "isic_id"] = new_isic_id

        print("Dataset balanced using SMOTE.\n New shape: ", self.df.shape)

    def _augment_images(self):
        """Function to augment images to balance the dataset."""
        target_0_count = len(self.df[self.df["target"] == 0])  # Count of target 0
        target_1_count = len(self.df[self.df["target"] == 1])  # Count of target 1
        print(f"Target 0 count: {target_0_count}, Target 1 count: {target_1_count}")

        # If target 0 count is greater than target 1 count, augment target 1 images
        if target_0_count > target_1_count:
            difference = target_0_count - target_1_count
            target_1_df = self.df[self.df["target"] == 1]
            augmented_images = []
            print(f"Adding {difference} augmented images to balance the dataset.")
# Augment images until the classes are equal
            while len(augmented_images) < difference:
                for idx, row in target_1_df.iterrows():
                    if len(augmented_images) >= difference:
                        break
                    isic_id = row["isic_id"]  # Get the ISIC ID
                    image_rgb = self._load_image_from_hdf5(isic_id)  # Load the image from HDF5
                    import torchvision.transforms.functional as F_v
                    image_pil = F_v.to_pil_image(image_rgb)  # Convert to PIL Image for transform
                    # Apply random augmentations to the images
                    augmentation_transforms = albumentations.Compose([
                        albumentations.Transpose(p=0.5),
                        albumentations.VerticalFlip(p=0.5),
                        albumentations.HorizontalFlip(p=0.5),
                        albumentations.RandomBrightnessContrast(limit=0.2, p=0.75),
                        albumentations.OneOf([
                            albumentations.MotionBlur(blur_limit=5),
                            albumentations.MedianBlur(blur_limit=5),
                            albumentations.GaussianBlur(blur_limit=5),
                            albumentations.GaussNoise(var_limit=(5.0, 30.0)),
                        ], p=0.7),
                        albumentations.OneOf([
                            albumentations.OpticalDistortion(distort_limit=1.0),
                            albumentations.GridDistortion(num_steps=5, distort_limit=1.0),
                            albumentations.ElasticTransform(alpha=3),
                        ], p=0.7),
                        albumentations.CLAHE(clip_limit=4.0, p=0.7),
                        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
                        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
                        albumentations.Normalize()
                    ])
                    augmented = augmentation_transforms(image=np.array(image_pil))
                    # Extract the transformed image from the output dictionary
                    augmented_image_np = augmented["image"]
                    _, encoded_image = cv2.imencode('.jpg', cv2.cvtColor(augmented_image_np, cv2.COLOR_RGB2BGR))
                    
                    new_isic_id = f"{isic_id}_aug_{len(augmented_images)}"  # Create a new ISIC ID
                    augmented_images.append((new_isic_id, encoded_image, row))
                ## break ##

            # Add augmented images to the existing HDF5 file and DataFrame
            with h5py.File(self.hdf5_path, "a") as hf:
                for new_isic_id, encoded_image, row in augmented_images:
                    hf.create_dataset(new_isic_id, data=encoded_image)
                    new_row = row.copy()
                    new_row["isic_id"] = new_isic_id
                    self.df = pd.concat(
                        [self.df, pd.DataFrame([new_row])], ignore_index=True
                    )
                    ## break ##
                hf.close()

            print(
                f"Added {len(augmented_images)} augmented images to balance the dataset."
            )
            print(f"New dataset shape: {self.df.shape}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        isic_id = row["isic_id"]

        # Load image from HDF5
        image_rgb = self._load_image_from_hdf5(isic_id)

        # Apply transforms (PIL-style transforms require converting np array to PIL, or we can do tensor transforms)
        if self.transform is not None:
            # Convert NumPy array (H x W x C) to a PIL Image
            import torchvision.transforms.functional as F_v

            image_pil = F_v.to_pil_image(image_rgb)
            image = self.transform(image_pil)
        else:
            # By default, convert it to a tensor (C x H x W)
            image = torch.from_numpy(image_rgb).permute(2, 0, 1).float()

        if self.is_labelled:
            label = row["target"]
            label = torch.tensor(label).float()
            return image, label, isic_id
        else:
            return image, isic_id

    def _load_image_from_hdf5(self, isic_id: str):
        """
        Loads and decodes an image from HDF5 by isic_id.
        Returns a NumPy array in RGB format (H x W x 3).
        """
        with h5py.File(self.hdf5_path, "r") as hf:
            encoded_bytes = hf[isic_id][()]  # uint8 array

        # Decode the image bytes with OpenCV (returns BGR)
        image_bgr = cv2.imdecode(encoded_bytes, cv2.IMREAD_COLOR)
        # Convert to RGB
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        return image_rgb


# ----------------------------------------------------
# 2. DataFrames and Basic Preprocessing / Transforms
# ----------------------------------------------------
# -----------------------------
# 3. Data Setup (Train/Valid/Test)
# -----------------------------
TRAIN_METADATA_CSV = "data/new-train-metadata.csv"
TEST_METADATA_CSV = "data/students-test-metadata.csv"
VALID_METADATA_CSV = "data/new-valid-metadata.csv"

TRAIN_HDF5 = "data/train-image.hdf5"
TEST_HDF5 = "data/test-image.hdf5"
VALID_HDF5 = "data/valid-image.hdf5"

train_df = pd.read_csv(TRAIN_METADATA_CSV)
train_df = train_df.sample(frac=0.02, random_state=42).reset_index(
    drop=True
)  # Sample 2% of the data
test_df = pd.read_csv(TEST_METADATA_CSV)
valid_df = pd.read_csv(VALID_METADATA_CSV)

print(f"train_df shape: {train_df.shape}")
print(f"valid_df shape: {valid_df.shape}")
print(f"test_df shape:  {test_df.shape}")


##
# Example: split train_df into 80% train / 20% valid
# train_size = int(0.8 * len(train_df))
# valid_size = len(train_df) - train_size
# train_subset, valid_subset = random_split(
#     train_df,
#     [train_size, valid_size],
#     generator=torch.Generator().manual_seed(42)
# )

# train_df_sub = train_df.iloc[train_subset.indices].reset_index(drop=True)
# valid_df_sub = train_df.iloc[valid_subset.indices].reset_index(drop=True)

# print(f"Train samples: {len(train_df_sub)}, Valid samples: {len(valid_df_sub)}")
##

print(f"Train samples: {len(train_df)}, Valid samples: {len(valid_df)}")

# Basic transforms for ResNet
resnet_transforms = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Create Datasets
train_dataset = ISIC_HDF5_Dataset(
    df=train_df,  # Change to train_df if using the original train_df
    hdf5_path=TRAIN_HDF5,
    transform=resnet_transforms,
    is_labelled=True
    #balanced=True,  # Balanced dataset for training
)

valid_dataset = ISIC_HDF5_Dataset(
    df=valid_df,  # Change to valid_df if using the original valid_df
    hdf5_path=VALID_HDF5,  # Change to VALID_HDF5 if using the original valid_df
    transform=resnet_transforms,
    is_labelled=True,
)

test_dataset = ISIC_HDF5_Dataset(
    df=test_df, hdf5_path=TEST_HDF5, transform=resnet_transforms, is_labelled=False
)


print("Created train/valid/test datasets.")


def store_validation_set(valid_dataset, valid_df_sub):
    # Store the images for valid_dataset in a file named valid-image.hdf5
    valid_hdf5_path = "data/valid-image.hdf5"
    with h5py.File(valid_hdf5_path, "w") as hf:
        for idx in range(len(valid_dataset)):
            image, label, isic_id = valid_dataset[idx]
            # Convert tensor to numpy array and encode as JPEG
            image_np = image.permute(1, 2, 0).numpy().astype(np.uint8)
            _, encoded_image = cv2.imencode(
                ".jpg", cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
            )
            hf.create_dataset(isic_id, data=encoded_image)

    # Store the dataframe for valid_dataset as new-valid-metadata.csv
    valid_df_sub.to_csv("data/new-valid-metadata.csv", index=False)
    print("Validation sets stored at data/.")


def store_training_set(train_dataset, train_df_sub):
    # Store the images for train_dataset in a file named train-image.hdf5
    train_hdf5_path = "data/train-image.hdf5"
    with h5py.File(train_hdf5_path, "w") as hf:
        for idx in range(len(train_dataset)):
            image, label, isic_id = train_dataset[idx]
            # Convert tensor to numpy array and encode as JPEG
            image_np = image.permute(1, 2, 0).numpy().astype(np.uint8)
            _, encoded_image = cv2.imencode(
                ".jpg", cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
            )
            hf.create_dataset(isic_id, data=encoded_image)

    # Store the dataframe for train_dataset as new-train-metadata.csv
    train_df_sub.to_csv("data/new-train-metadata.csv", index=False)
    print("Training sets stored at data/.")


# store_validation_set(valid_dataset,valid_df_sub)
# store_training_set(train_dataset,train_df_sub)

Using device: cpu


/tmp/ipykernel_92987/1963830541.py:261: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(TRAIN_METADATA_CSV)
/tmp/ipykernel_92987/1963830541.py:266: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv(VALID_METADATA_CSV)


train_df shape: (320767, 55)
valid_df shape: (80192, 55)
test_df shape:  (100, 44)
Train samples: 320767, Valid samples: 80192
Target 0 count: 320499, Target 1 count: 268
Adding 320231 augmented images to balance the dataset.


/tmp/ipykernel_92987/1963830541.py:161: UserWarning: Argument 'limit' is not valid and will be ignored.
  albumentations.RandomBrightnessContrast(limit=0.2, p=0.75),
/home/rribes/.local/lib/python3.8/site-packages/pydantic/main.py:214: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
[ WARN:0@2341.387] global loadsave.cpp:1329 imencode Unsupported depth image for selected encoder is fallbacked to CV_8U.


In [26]:
# -----------------------------
# 4. RandomSampler for 1000 Samples per Epoch
# -----------------------------
# Instead of weighting for class imbalance, we simply draw 1000 random samples each epoch.
# Students can discover the imbalance issue themselves!

from torch.utils.data import RandomSampler

# A RandomSampler with replacement=True can pick num_samples=1000 each epoch
sampler = RandomSampler(data_source=train_dataset, replacement=True, num_samples=1000)

BATCH_SIZE = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler,  # No shuffle needed, RandomSampler handles randomness
    num_workers=2,
)

valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)

test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)

print(
    f"Train loader: {len(train_loader)} batches (total = 1000 samples / batch_size = {BATCH_SIZE})"
)
print(f"Valid loader: {len(valid_loader)} batches")
print(f"Test loader:  {len(test_loader)} batches")

Train loader: 125 batches (total = 1000 samples / batch_size = 8)
Valid loader: 10024 batches
Test loader:  13 batches


In [ ]:
# -----------------------------
# 5. Load & Modify ResNet50 for Binary Classification
# -----------------------------
import torch.optim as optim
from torchvision import models

model = models.resnet50(pretrained=True)

# Replace final layer for binary classification
model.fc = nn.Linear(in_features=2048, out_features=1)

model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/home/rribes/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rribes/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## DINO v2

In [27]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('/home/rribes/.cache/kagglehub/models/metaresearch/dinov2/pyTorch/base/1') #hardcoded
model = AutoModel.from_pretrained('/home/rribes/.cache/kagglehub/models/metaresearch/dinov2/pyTorch/base/1') #hardcoded

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [5]:
# -----------------------------
# 6. Training Loop (5 Epochs, Only 1000 Samples/Epoch)
# -----------------------------

import torch.optim as optim

model.fc = nn.Linear(in_features=2048, out_features=1)

model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Unfreeze all layers for fine-tuning (if desired)
for param in model.parameters():
    param.requires_grad = True

EPOCHS = 5
for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0

    for images, labels, _ in tqdm(valid_loader, desc=f"Epoch {epoch}", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(images).view(-1)  # [batch_size]

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    print(f"Epoch {epoch}/{EPOCHS} | Train Loss: {avg_train_loss:.4f}))")

Epoch 1:   0%|          | 0/10024 [00:00<?, ?it/s]

: 

## ResNet18

In [ ]:
#import kagglehub

# Download latest version
#path = kagglehub.model_download("xukongji/resnet18/pyTorch/resnet18")

#print("Path to model files:", path)


  0%|          | 0.00/42.7M [00:00<?, ?B/s]






























































100%|██████████| 42.7M/42.7M [00:02<00:00, 15.2MB/s]



























































100%|██████████| 42.7M/42.7M [00:03<00:00, 11.6MB/s]
































































































































100%|██████████| 42.7M/42.7M [00:05<00:00, 7.47MB/s]












100%|██████████| 42.7M/42.7M [00:06<00:00, 7.06MB/s]


100%|██████████| 42.7M/42.7M [00:06<00:00, 7.04MB/s]

Path to model files: /home/rribes/.cache/kagglehub/models/xukongji/resnet18/pyTorch/resnet18/1


In [ ]:
# Load the pretrained model (assumes it was saved as a complete model)
import os
import glob
import torch
from torchvision import models
import torch.optim as optim
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Retrieve all .pt files from the specified folder and choose one (the latest in this example)
path = "/home/rribes/.cache/kagglehub/models/xukongji/resnet18/pyTorch/resnet18/1/resnet18_baseline_112/" # hardcoded
pt_files = glob.glob(os.path.join(path, '*.pt'))
pt_files.sort()  # sort for consistency
checkpoint_path = pt_files[-1]

model_ft = torch.load(checkpoint_path, map_location=torch.device("cpu"))

# Modify the final layer for binary classification
model_ft_temp = models.resnet18()
model_ft_temp.load_state_dict(model_ft, strict=False)
num_features = model_ft_temp.fc.in_features
model_ft = model_ft_temp
model_ft.fc = nn.Linear(num_features, 1)

model_ft = model_ft.to(device=device)

criterion_ft = torch.nn.BCEWithLogitsLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4)

EPOCHS = 5  # or any number you prefer
for epoch in range(1, EPOCHS + 1):
    model_ft.train()
    running_loss = 0.0

    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer_ft.zero_grad()
        logits = model_ft(images).view(-1)
        loss = criterion_ft(logits, labels)
        loss.backward()
        optimizer_ft.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}/{EPOCHS} | Loss: {avg_loss:.4f}")

Epoch 1/5 | Loss: 0.0602
Epoch 2/5 | Loss: 0.0218
Epoch 3/5 | Loss: 0.0106
Epoch 4/5 | Loss: 0.0101
Epoch 5/5 | Loss: 0.0015


In [32]:
# -----------------------------
# 7. Inference on Test Set & Submission
# -----------------------------
model = model_ft
model.eval()
predictions = []

with torch.no_grad():
    for images, isic_ids in tqdm(test_loader, desc="Inference on Test"):
        images = images.to(device)
        logits = model(images).view(-1)  # shape [batch_size]
        probs = torch.sigmoid(logits)  # shape [batch_size], in [0,1]

        probs = probs.cpu().numpy()

        for isic_id, p in zip(isic_ids, probs):
            predictions.append({"isic_id": isic_id, "target": float(p)})

submission_df = pd.DataFrame(predictions)
submission_df = submission_df.sort_values(by="isic_id").reset_index(drop=True)

submission_file = "submission.csv"
submission_df.to_csv(submission_file, index=False)

print(f"Saved submission with {len(submission_df)} rows to {submission_file}")
display(submission_df.head(10))

Inference on Test: 100%|██████████| 13/13 [00:03<00:00,  3.84it/s]


Saved submission with 100 rows to submission.csv


,isic_id,target
0,ISIC_0082829,0.001589
1,ISIC_0114227,0.000549
2,ISIC_0157465,0.001845
3,ISIC_0197356,0.000336
4,ISIC_0275647,0.000408
5,ISIC_0332355,0.000520
6,ISIC_0528190,0.000398
7,ISIC_0576478,0.002629
8,ISIC_0719839,0.000418
9,ISIC_0968965,0.000521
